In [1]:
import numpy as np
from numpy import cos, exp, pi, sin, sqrt


用不动点迭代法

<!-- x1_new = 1/3 * (cos(x2*x3) + 0.5)
x2_new = -0.1 +- 1/9 * sqrt(x1**2 + sin(x3) + 1.06)
x3_new = 1/20 * (1 - 10/3*pi - exp(-x1*x2)) -->

若 $x_2 \ge -0.1$, 采用
$$
    \left\{
    \begin{align*}
        x_1 &= \dfrac{1}{3} \left( \cos(x_2 x_3) + 0.5 \right) \\
        x_2 &= -0.1 + \dfrac{1}{9} \sqrt{x_1^2 + \sin(x_3) + 1.06} \\
        x_3 &= \dfrac{1}{20} \left( 1 - \dfrac{10}{3} \pi - e^{-x_1 x_2} \right)
    \end{align*}
    \right.
$$

若 $x_2 \le -0.1$, 采用
$$
    \left\{
    \begin{align*}
        x_1 &= \dfrac{1}{3} \left( \cos(x_2 x_3) + 0.5 \right) \\
        x_2 &= -0.1 - \dfrac{1}{9} \sqrt{x_1^2 + \sin(x_3) + 1.06} \\
        x_3 &= \dfrac{1}{20} \left( 1 - \dfrac{10}{3} \pi - e^{-x_1 x_2} \right)
    \end{align*}
    \right.
$$

这两个迭代公式都是 $[-1, 1]^3$ 上的压缩映射，因此都是收敛的。

In [2]:
def f0(x):
    # 假设 x2 >= -0.1
    x1, x2, x3 = x
    x1_new = 1/3 * (cos(x2*x3) + 0.5)
    x2_new = -0.1 + 1/9 * sqrt(x1**2 + sin(x3) + 1.06)
    x3_new = 1/20 * (1 - 10/3*pi - exp(-x1*x2))
    return np.array([x1_new, x2_new, x3_new])


def f1(x):
    # 假设 x2 <= -0.1
    x1, x2, x3 = x
    x1_new = 1/3 * (cos(x2*x3) + 0.5)
    x2_new = -0.1 - 1/9 * sqrt(x1**2 + sin(x3) + 1.06)
    x3_new = 1/20 * (1 - 10/3*pi - exp(-x1*x2))
    return np.array([x1_new, x2_new, x3_new])


def solve(f, x, tol):
    x = np.array(x)
    while True:
        x_new = f(x)
        if np.linalg.norm(x_new - x) < tol:
            return x_new
        x = x_new


In [3]:
x1, x2, x3 = solve(f0, (0, 0, 0), 1e-8)
print(f"x1 = {x1:.6f}, x2 = {x2:.6f}, x3 = {x3:.6f}")
x1, x2, x3 = solve(f1, (0, 0, 0), 1e-8)
print(f"x1 = {x1:.6f}, x2 = {x2:.6f}, x3 = {x3:.6f}")


x1 = 0.500000, x2 = 0.000000, x3 = -0.523599
x1 = 0.498145, x2 = -0.199606, x3 = -0.528826


用牛顿法，Jacobi 矩阵是手动求的，并非数值微分。

In [4]:
def f(x):
    x1, x2, x3 = x
    return np.array([
        3*x1 - cos(x2*x3) - 1/2,
        x1**2 - 81*(x2+0.1)**2 + sin(x3) + 1.06,
        exp(-x1*x2) + 20*x3 + 10/3*pi - 1,
    ])


def jacobian(x):
    x1, x2, x3 = x
    return np.array([
        [3, x3*sin(x2*x3), x2*sin(x2*x3)],
        [2*x1, -162*(x2+0.1), cos(x3)],
        [-x2*exp(-x1*x2), -x1*exp(-x1*x2), 20],
    ])


def solve(f, jacobian, x, tol):
    x = np.array(x)
    while True:
        delta_x = np.linalg.solve(jacobian(x), -f(x))
        x = x + delta_x
        x1, x2, x3 = x
        if np.linalg.norm(delta_x) < tol:
            return x1, x2, x3


x1, x2, x3 = solve(f, jacobian, (0, 0, 0), 1e-8)
print(f"初值为 (0, 0, 0) 时：")
print(f"x1 = {x1:.6f}, x2 = {x2:.6f}, x3 = {x3:.6f}")
x1, x2, x3 = solve(f, jacobian, (6, -6, 10), 1e-8)
print(f"初值为 (6, -6, 10) 时：")
print(f"x1 = {x1:.6f}, x2 = {x2:.6f}, x3 = {x3:.6f}")
x1, x2, x3 = solve(f, jacobian, (-6, 6, -10), 1e-8)
print(f"初值为 (-6, 6, -10) 时：")
print(f"x1 = {x1:.6f}, x2 = {x2:.6f}, x3 = {x3:.6f}")


初值为 (0, 0, 0) 时：
x1 = 0.500000, x2 = -0.000000, x3 = -0.523599
初值为 (6, -6, 10) 时：
x1 = 0.498145, x2 = -0.199606, x3 = -0.528826
初值为 (-6, 6, -10) 时：
x1 = 0.498145, x2 = -0.199606, x3 = -0.528826
